In [141]:
import pandas as pd
import numpy as np
import matplotlib as plt
import math
%matplotlib notebook

In [128]:
file = "./data/SimulatedData.xlsx"
data = pd.read_excel(file, sheet_name=0)
data["NumOfWeek"] = (data["Season"]-1)*12 + data["Week"]

In [129]:
remain_stock = {"A":10, "B":10, "C":10}
week_tree = {0:{"Week":0, "Child": [], "RemainStock":remain_stock, "Type": "D", "MaxPrice":999, "n":0, "V": 0}}
print(week_tree)
len(week_tree)

{0: {'Week': 0, 'Child': [], 'RemainStock': {'A': 10, 'B': 10, 'C': 10}, 'Type': 'D', 'MaxPrice': 999, 'n': 0, 'V': 0}}


1

In [130]:
# add state-of-nature nodes to the tree 
def expand_tree(tree, node):
    if len(tree[node]["Child"]) == 0:
        # set the price no largert than the previous bid
        for price in range(99, tree[node]["MaxPrice"]+1, 100):
            tree[len(tree)] = {"Week": tree[node]["Week"], "Child":[], "Parent": node, "Price": price, 
                                 "Type": "S", "n":0, "V": 0, "UCB": float('inf')}
            tree[node]["Child"].append(len(tree)-1)
            
expand_tree(week_tree, 0)
wt = pd.DataFrame(week_tree)
wt = wt.T
wt

,Child,MaxPrice,Parent,Price,RemainStock,Type,UCB,V,Week,n
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",999,NaN,NaN,"{'A': 10, 'B': 10, 'C': 10}",D,NaN,0,0,0
1,[],NaN,0,99,NaN,S,inf,0,0,0
2,[],NaN,0,199,NaN,S,inf,0,0,0
3,[],NaN,0,299,NaN,S,inf,0,0,0
4,[],NaN,0,399,NaN,S,inf,0,0,0
5,[],NaN,0,499,NaN,S,inf,0,0,0
6,[],NaN,0,599,NaN,S,inf,0,0,0
7,[],NaN,0,699,NaN,S,inf,0,0,0
8,[],NaN,0,799,NaN,S,inf,0,0,0
9,[],NaN,0,899,NaN,S,inf,0,0,0


In [131]:
test_tree = {0:{"Week":0, "Child": [1,2,3,4,5], "RemainStock":remain_stock, "Type": "D","MaxPrice":999, "n":0, "V": 0},
1:{"Week": 0, "Child":[], "Parent": 0, "Price": 99, "Type": "S", "n":0, "V": 0, "UCB": 1.1},
2:{"Week": 0, "Child":[], "Parent": 0, "Price": 199, "Type": "S", "n":0, "V": 0, "UCB": 2.2},
3:{"Week": 0, "Child":[], "Parent": 0, "Price": 299, "Type": "S", "n":0, "V": 0, "UCB": 3.3},
4:{"Week": 0, "Child":[], "Parent": 0, "Price": 399, "Type": "S", "n":0, "V": 0, "UCB": 4.4},
5:{"Week": 0, "Child":[], "Parent": 0, "Price": 499, "Type": "S", "n":0, "V": 0, "UCB": 5.5}}

In [132]:
def get_max_child_ucb(tree, node):
    if len(tree[node]["Child"]) == 0:
        expand_tree(tree, node) 
    max_UCB = float("-inf")
    max_node = 0
    for child in tree[node]["Child"]:
        if tree[child]["UCB"] > max_UCB:
            max_UCB = tree[child]["UCB"]
            max_node = child
    return child

def get_random_child_ucb(tree, node):
    if len(tree[node]["Child"]) == 0:
        expand_tree(tree, node) 
    import random
    return tree[node]["Child"][int(random.random() * len(tree[node]["Child"]))]
    
    
    
get_max_child_ucb(test_tree, 0)

5

In [133]:
def do_back_prop(tree, node, pointer, revenue):
    rollout = revenue
    while pointer > node:
        pointer = tree[pointer]["Parent"]
        # Update V and n
        tree[pointer]["V"] = (tree[pointer]["V"] * tree[pointer]["n"] + rollout) / (tree[pointer]["n"] + 1)
        tree[pointer]["n"] += 1
        # Only update UCB for a state-of-nature node
        # Also update the N_i and UCB for options not chosen in this simulation
        if tree[pointer]["Type"] == "S":
            tree[pointer]["UCB"] = tree[pointer]["V"] + 2 * (
                math.log(tree[tree[pointer]["Parent"]]["n"] + 1) / tree[pointer]["n"]) ** 0.5
            peer_list = tree[tree[pointer]["Parent"]]["Child"]
            for no in peer_list:
                if pointer == no: # myself
                    continue
                if tree[no]["n"] > 0:
                    tree[no]["UCB"] = tree[no]["V"] + 2 * (math.log(tree[tree[no]["Parent"]]["n"] + 1)
                     / tree[no]["n"]) ** 0.5

In [134]:
def get_revenue(remain_stock, price, data_week, last_week_price):
    revenue = 0
    for SKU in ["A", "B", "C"]:
        for i in range(7):
            value_i = data_week[data_week["SKU"] == SKU].iloc[:, 3+i].item()
            if value_i > price and remain_stock[SKU] > 0:
                remain_stock[SKU] -= 1
                revenue += price
            else:
                return_i = data_week[data_week["SKU"] == SKU].iloc[:, 10+i].item()
                if return_i > 0:
                    last_week_price[SKU].append(return_i)
    return revenue

def get_last_week_revenue(remain_stock, price, data_week, last_week_price):
    revenue = 0
    for SKU in ["A", "B", "C"]:
        for i in range(7):
            value_i = data_week[data_week["SKU"] == SKU].iloc[:, 3+i].item()
            if value_i > price and remain_stock[SKU] > 0:
                remain_stock[SKU] -= 1
                revenue += price
        for value in last_week_price[SKU]:
            if value > price and remain_stock[SKU] > 0:
                remain_stock[SKU] -= 1
                revenue += price
    return revenue
    
remain_stock = {"A":0, "B":10, "C":1}
last_week_price = {"A":[], "B":[], "C":[]}
print(get_revenue(remain_stock, 899, data[data["NumOfWeek"] == 1], last_week_price))
print(remain_stock)
print(last_week_price)

899
{'A': 0, 'B': 10, 'C': 0}
{'A': [], 'B': [231.8298749926122], 'C': [137.59269362478472]}


In [135]:
def check_sold_out(remain_stock):
    if remain_stock["A"] == 0 and remain_stock["B"] == 0 and remain_stock["C"] == 0:
        return True
    return False
check_sold_out(remain_stock = {"A":0, "B":0, "C":0})

True

* Tree: the tree selected to be built
* Node: the node to be started with;
* k: the number of simulations

In [150]:
def build_tree(tree, node, k=500): 
    loop = 0 # total 600 weeks -> 50 loops in total
    for t in range(k):
        if t % 1000 == 1:
            print("executed %i loops" % t)
        loop %= 50
        loop += 1
        start_week = (loop - 1) * 12 + 1
        end_week = loop * 12
        
        week = start_week
        pointer = node
        revenue = 0
        remain_stock = {"A":10, "B":10, "C":10}
        last_week_price = {"A":[], "B":[], "C":[]}
        # simulate from the 1st week to 11th week
        while week <= end_week:
            # expanson
            if len(tree[pointer]["Child"]) == 0:
                expand_tree(tree, pointer)
            # pointer move to S type node
            pointer = get_max_child_ucb(tree, pointer)
            price = tree[pointer]["Price"]
            if week != end_week:
                revenue += get_revenue(remain_stock, price, data[data["NumOfWeek"] == week], last_week_price)
            else:
                # simulate the final (12th) week
                revenue += get_last_week_revenue(remain_stock, price, data[data["NumOfWeek"] == week], last_week_price)
            if check_sold_out(remain_stock):
                break 
            week += 1
            # pointer move to D type node
            # Check if the state has been realized before
            exist_flag = 0
            for child_id in tree[pointer]["Child"]:
                if tree[child_id]["RemainStock"] == remain_stock:
                    exist_flag = 1
                    pointer = child_id
                    break
            # this state is not realized before
            if exist_flag == 0:
                tree[len(tree)]= {"Week": week, "Child": [], "Parent": pointer, "RemainStock":remain_stock, "Type": "D", "MaxPrice":price, "n":0, "V": 0}
                tree[pointer]["Child"].append(len(tree)-1)
                pointer = len(tree)-1
                
        # Start backpropagation; BSR is the total revenue going forward in the simulation
        do_back_prop(tree, node, pointer, revenue)


In [ ]:
week_tree = {0:{"Week":0, "Child": [], "RemainStock":remain_stock, "Type": "D", "MaxPrice":999, "n":0, "V": 0}}
build_tree(week_tree, 0, 100000)
pd_tree = pd.DataFrame(week_tree).T
pd_tree

executed 1 loops
executed 1001 loops
executed 2001 loops
executed 3001 loops
executed 4001 loops
executed 5001 loops
executed 6001 loops
executed 7001 loops
executed 8001 loops
executed 9001 loops
executed 10001 loops
executed 11001 loops
executed 12001 loops
executed 13001 loops
executed 14001 loops
executed 15001 loops
executed 16001 loops
executed 17001 loops
executed 18001 loops
executed 19001 loops
executed 20001 loops
executed 21001 loops
executed 22001 loops
executed 23001 loops
executed 24001 loops
executed 25001 loops
executed 26001 loops
executed 27001 loops
executed 28001 loops
executed 29001 loops
executed 30001 loops
executed 31001 loops
executed 32001 loops
executed 33001 loops
executed 34001 loops
executed 35001 loops
executed 36001 loops
executed 37001 loops
executed 38001 loops
executed 39001 loops
executed 40001 loops
executed 41001 loops
executed 42001 loops
executed 43001 loops
executed 44001 loops
executed 45001 loops
executed 46001 loops
executed 47001 loops
execu

In [ ]:
pd_tree[pd_tree["MaxPrice"] ==899]

In [72]:
data_week = data[data["NumOfWeek"] == 1]
data_week

,Season,Week,SKU,ValueB1,ValueB2,ValueB3,ValueB4,ValueB5,ValueB6,ValueB7,ReturnB1,ReturnB2,ReturnB3,ReturnB4,ReturnB5,ReturnB6,ReturnB7,NumOfWeek
0,1,1,A,838.350764,781.069723,563.269441,141.161300,0.000000,969.215317,884.195936,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1
12,1,1,B,0.000000,391.180312,0.000000,784.083351,184.657584,698.995915,0.000000,0.0,0.0,0.0,0.000000,0.0,231.829875,0.0,1
24,1,1,C,470.584865,909.715560,77.849248,909.432055,77.443591,246.156484,0.000000,0.0,0.0,0.0,137.592694,0.0,0.000000,0.0,1


In [82]:

a = data_week[data_week["SKU"] == "A"].iloc[:, 3:17]
a

,ValueB1,ValueB2,ValueB3,ValueB4,ValueB5,ValueB6,ValueB7,ReturnB1,ReturnB2,ReturnB3,ReturnB4,ReturnB5,ReturnB6,ReturnB7
0,839.350764,782.069723,564.269441,142.1613,1.0,970.215317,885.195936,1.0,1.0,1.0,1.0,1.0,1.0,1.0
